In [1]:
import pandas as pd 
import numpy as np
import numpy as np 
import pickle
import itertools

import matplotlib.pyplot as plt
import seaborn as sns

import spacy
import re
import nltk, string
from nltk.corpus import stopwords
from textblob import TextBlob, Word

from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import hstack, vstack
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, GridSearchCV
from sklearn.model_selection import learning_curve, StratifiedShuffleSplit, StratifiedKFold, ShuffleSplit

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import recall_score, accuracy_score, f1_score
#load stopwords corpus
import nltk
nltk.download('stopwords')
#load the en_core_web_sm 
import en_core_web_sm
nlp = en_core_web_sm.load()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\leonayu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\leonayu\AppData\Local\Continuum\anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
C:\Users\leonayu\AppData\Local\Continuum\anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model wi

In [3]:
CBS_NEWS=pd.read_csv(r"C:\\Users\\leonayu\\Documents\\Zhuohan\\725\\2nd_semester\\data\\cbs_news_content.csv",header=None)
CBS_NEWS.columns=['link',"title","Author","timestamp","Paragraph_num","Paragraph_text","download_date"]

In [4]:
CBS_NEWS.head(2)

,link,title,Author,timestamp,Paragraph_num,Paragraph_text,download_date
0,https://www.cbsnews.com/news/antares-rocket-bo...,Northrop Grumman Antares rocket boosts space s...,WILLIAM HARWOOD,2/15/2020 16:26,0,"Running nearly a week late, a Northrop Grumman...",2/17/2020 21:19
1,https://www.cbsnews.com/news/antares-rocket-bo...,Northrop Grumman Antares rocket boosts space s...,WILLIAM HARWOOD,2/15/2020 16:26,1,The rocket's Russian-built RD-181 first stage ...,2/17/2020 21:19


In [5]:
#create a stop words list
stopWordList = [word for word in list(stopwords.words('english') ) if word not in ['no', 'not']]
# all the data in this dataset is true news, then the target show all equal to 0
CBS_NEWS["target"]=0

In [6]:
# list of punctuation that need to be removed
punct_removal = list(string.punctuation) + ["--", "---"]
# define the the word pattern
word_pattern = r"(?u)\b\w\w+\b"

In [7]:
#check if NAN in columns
CBS_NEWS["Paragraph_text"].isnull().values.any()

#check how many of NAN in columns
CBS_NEWS["Paragraph_text"].isnull().sum()
# check how many NAN in the whole dataset
CBS_NEWS.isnull().values.sum()

# fill the NAN with whitespace
CBS_NEWS["Paragraph_text"]=CBS_NEWS["Paragraph_text"].fillna(" ")

In [8]:
%%time
tokened_para=[]
for i in range(len(CBS_NEWS)):
    word_pat = re.compile(word_pattern, flags = re.UNICODE )

    CBS_NEWS["Paragraph_text"][i] = " ".join(word_pat.findall(CBS_NEWS["Paragraph_text"][i])).lower()
    
    tokens = nlp(CBS_NEWS["Paragraph_text"][i])
    #lemmatising tokens 
    tokens = [t.lemma_.strip() if t.lemma_ != "-PRON-" else t.lower_ for t in tokens]
    #stopword and punctuation removal
    tokens = [t for t in tokens if (t not in stopWordList and t not in punct_removal)]
    processed_text = " ".join(tokens)
    tokened_para.append(processed_text) 
    

C:\Users\leonayu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Wall time: 8min 42s


In [9]:
%%time
tokened_title=[]
for i in range(len(CBS_NEWS)):
    word_pat = re.compile(word_pattern, flags = re.UNICODE )
    CBS_NEWS["title"][i] = " ".join(word_pat.findall(CBS_NEWS["title"][i])).lower()
    tokens = nlp(CBS_NEWS["title"][i])
    #lemmatising tokens 
    tokens = [t.lemma_.strip() if t.lemma_ != "-PRON-" else t.lower_ for t in tokens]
    #stopword and punctuation removal
    tokens = [t for t in tokens if (t not in stopWordList and t not in punct_removal)]
    processed_text = " ".join(tokens)
    tokened_title.append(processed_text) 

C:\Users\leonayu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Wall time: 8min


In [10]:
tokened_para_df=pd.DataFrame(tokened_para)
tokened_title_df=pd.DataFrame(tokened_title)
CBS_NEWS["tokened_para"]=tokened_para_df
CBS_NEWS["tokened_title"]=tokened_title_df
CBS_NEWS['all_text']=CBS_NEWS["tokened_title"]+CBS_NEWS["tokened_para"]

In [11]:
# CBS_NEWS[["tokened_title","tokened_para","target"]]
CBS_NEWS['all_text']

0       northrop grumman antares rocket boost space st...
1       northrop grumman antares rocket boost space st...
2       northrop grumman antares rocket boost space st...
3       northrop grumman antares rocket boost space st...
4       northrop grumman antares rocket boost space st...
5       northrop grumman antares rocket boost space st...
6       northrop grumman antares rocket boost space st...
7       northrop grumman antares rocket boost space st...
8       northrop grumman antares rocket boost space st...
9       northrop grumman antares rocket boost space st...
10      northrop grumman antares rocket boost space st...
11      northrop grumman antares rocket boost space st...
12      northrop grumman antares rocket boost space st...
13      northrop grumman antares rocket boost space st...
14      northrop grumman antares rocket boost space st...
15      northrop grumman antares rocket boost space st...
16      northrop grumman antares rocket boost space st...
17      spacex

In [13]:
%%time
vec = TfidfVectorizer(ngram_range=(1,3))
vec.fit(CBS_NEWS.all_text)
vocab = vec.vocabulary_
    
vecH = TfidfVectorizer(ngram_range=(1,3), max_df=0.8, vocabulary=vocab)
vecH.fit(CBS_NEWS.tokened_title.unique())
    
vecB = TfidfVectorizer(ngram_range=(1,3), max_df=0.8, vocabulary=vocab)
vecB.fit(CBS_NEWS.tokened_para.unique())
    
xHTrainTfidf = vecH.transform(CBS_NEWS.tokened_title)
print ('Train headline matrix shape: \t', xHTrainTfidf.shape)
    
xBTrainTfidf = vecB.transform(CBS_NEWS.tokened_para)
print('Train body matrix shape: \t', xBTrainTfidf.shape) 

Train headline matrix shape: 	 (9439, 407770)
Train body matrix shape: 	 (9439, 407770)
